To construct the Decision tree using the training data sets under supervised learning concept. Program: Write a program to demonstrate the working of the decision tree based ID3 algorithm. Use an appropriate data set for building the decision tree and apply this knowledge to classify a new sample.

In [8]:
import math
import csv

In [11]:
def load_csv(filename):
    """Load CSV file and return dataset and headers."""
    with open(filename, "r") as file:
        lines = csv.reader(file)
        dataset = list(lines)
        headers = dataset.pop(0)
    return dataset, headers

In [12]:
class Node:
    """Class representing a node in the decision tree."""
    def __init__(self, attribute):
        self.attribute = attribute
        self.children = []
        self.answer = ""

In [9]:
def subtables(data, col, delete):
    """Create subtables for each unique value of a column."""
    dic = {}
    coldata = [row[col] for row in data]
    attr = list(set(coldata))
    counts = [0] * len(attr)
    r = len(data)
    c = len(data[0])
    for x in range(len(attr)):
        for y in range(r):
            if data[y][col] == attr[x]:
                counts[x] += 1
    for x in range(len(attr)):
        dic[attr[x]] = [[0 for _ in range(c)] for _ in range(counts[x])]
        pos = 0
        for y in range(r):
            if data[y][col] == attr[x]:
                if delete:
                    del data[y][col]
                dic[attr[x]][pos] = data[y]
                pos += 1
    return attr, dic

In [ ]:
def entropy(S):
    """Calculate the entropy of a set."""
    attr = list(set(S))
    if len(attr) == 1:
        return 0
    counts = [0, 0]
    for i in range(2):
        counts[i] = sum([1 for x in S if attr[i] == x]) / (len(S) * 1.0)
    sums = 0
    for cnt in counts:
        sums += -1 * cnt * math.log(cnt, 2)
    return sums

In [13]:
def compute_gain(data, col):
    """Compute the information gain of a column."""
    attr, dic = subtables(data, col, delete=False)
    total_size = len(data)
    entropies = [0] * len(attr)
    ratio = [0] * len(attr)
    total_entropy = entropy([row[-1] for row in data])
    for x in range(len(attr)):
        ratio[x] = len(dic[attr[x]]) / (total_size * 1.0)
        entropies[x] = entropy([row[-1] for row in dic[attr[x]]])
        total_entropy -= ratio[x] * entropies[x]
    return total_entropy



In [14]:
def build_tree(data, features):
    """Build the decision tree using the ID3 algorithm."""
    lastcol = [row[-1] for row in data]
    if len(set(lastcol)) == 1:
        node = Node("")
        node.answer = lastcol[0]
        return node
    n = len(data[0]) - 1
    gains = [0] * n
    for col in range(n):
        gains[col] = compute_gain(data, col)
    split = gains.index(max(gains))
    node = Node(features[split])
    fea = features[:split] + features[split + 1:]
    attr, dic = subtables(data, split, delete=True)
    for x in range(len(attr)):
        child = build_tree(dic[attr[x]], fea)
        node.children.append((attr[x], child))
    return node

In [15]:
def print_tree(node, level=0):
    """Print the decision tree."""
    if node.answer != "":
        print(" " * level, node.answer)
        return
    print(" " * level, node.attribute)
    for value, n in node.children:
        print(" " * (level + 1), value)
        print_tree(n, level + 2)

In [16]:
def classify(node, x_test, features):
    """Classify a new sample using the decision tree."""
    if node.answer != "":
        print(node.answer)
        return
    pos = features.index(node.attribute)
    for value, n in node.children:
        if x_test[pos] == value:
            classify(n, x_test, features)

In [17]:
if __name__ == "__main__":
    # Load training data
    dataset, features = load_csv("data.csv")
    
    # Build decision tree
    tree = build_tree(dataset, features)
    print("The decision tree for the dataset using the ID3 algorithm is:")
    print_tree(tree)
    
    # Load test data
    test_data, test_features = load_csv("data_test.csv")
    for test_instance in test_data:
        print("The test instance:", test_instance)
        print("The label for the test instance:", end=" ")
        classify(tree, test_instance, test_features)

The decision tree for the dataset using the ID3 algorithm is:
 Outlook
  Rain
   Wind
    Strong
     No
    Weak
     Yes
  Sunny
   Humidity
    High
     No
    Normal
     Yes
  Overcast
   Yes
The test instance: ['Sunny', 'Hot', 'High', 'Weak']
The label for the test instance: No
The test instance: ['Rain', 'Cool', 'Normal', 'Strong']
The label for the test instance: No
The test instance: ['Overcast', 'Mild', 'High', 'Weak']
The label for the test instance: Yes
The test instance: ['Sunny', 'Cool', 'High', 'Strong']
The label for the test instance: No
